# OOP: Property Function and Decorator in Python

Let's say we are going to define a new class to describe person. 

To make it simple, the person will only have two attributes, the name, and the age.

### Implementation 1:

> we start simple on the first implementation, we use the `__init__()` method to initilise every new instances of the class. 
>
> - `name` is a mandatory parameter for a Person.
> - `age` is an optional parameter with a default value of zero. 

But this oversimplified implementation has a few serious flaws:

1. `name` and `age` parameter can take any data type, which does not make sense. 
2. there is no validation for `age` parameter, so age could be set to a ridiculous number, say 1000.

In [1]:
class Person:
    def __init__(self, name, age=0):
        self.name = name
        self.age = age
    
    def __str__(self):
        return f"{self.name}(Age: {self.age})"

p1 = Person("Mike")
print(p1)

p1.age = 23
print(p1.age)

Mike(Age: 0)
23


### Implementation 2:

> This time, we made some significant improvement. 
> Now, we will check if the `name` and `age` parameters are valid before initilising them. 

But still there is an issue:

- We are only able to catch the error when we create a new instance. If we have created a valid instance of Person, and then we can update the values of its attributes freely without going through any validation process. This is because the `__init__` function, where the validation code lives, is not called when we try to update attributes for an existing instance.

In [2]:
class Person:
    def __init__(self, name, age=0):
        # Initialise Person.name
        if type(name) == str:
            self.name = name
        else:
            print("Person.name can be String only.")
        
        # Initialise Person._age
        if type(age) not in (int, float):
            print("Age can be integer or float only.")
        elif age < 0:
            print("Age cannot less than zero.")
        else:
            self.age = age
        
    def __str__(self):
        return f"{self.name}(Age: {self.age})"
    
p1 = Person("Mike")
print(p1)
p2 = Person("Jack", "22")


p1.age = 23
print(p1.age)

# this implementation will only catch the error for new initiation
# hence it won't catch the following error
p1.age = "HAHA"
print(p1)

Mike(Age: 0)
Age can be integer or float only.
23
Mike(Age: HAHA)


### Implementation 3:

> This time, we will be using the python built-in function `property()` to help us addressing the issue we encountered above. 

Important changes:

1. At line 10, we are calling the method `self.set_age()` to do the age validation and set the value for `self._age` instead of `self.age`. Using an underscore at the beginning of an attribute/ variable name does not technically make any difference in python, but a convention to represent that this is supposed to be a private attribute which anyone should not access it directly. 
2. At line 12, 15, and 18, we defined 3 methods to get, set, delete the `self._age` attribute respectively. 
3. At line 31, we are using the `property()` to construct a property called `age`, which you can access using `Person.age`. By passing the 3 methods to the `property` function, we are telling Python that the `getter`, `setter`, and `deleter` for the property `age` is `get_age`, `set_age`, and `del_age` respectively.


In [3]:
class Person:
    def __init__(self, name, age=0):
        # Initialise Person.name
        if type(name) == str:
            self.name = name
        else:
            print("Person.name can be String only.")
        
        # call set age
        self.set_age(age)
        
    def __str__(self):
        return f"{self.name}(Age: {self._age})"
    
    def get_age(self):
        return self._age
    
    def set_age(self, value):
        if type(value) not in (int, float):
            print("Age can be integer or float only.")
            self._age = None
        elif value < 0:
            print("Age cannot less than zero.")
            self._age = None
        else:
            self._age = value
    
    def del_age(self):
        self._age = None
            
    age = property(get_age, set_age, del_age)
    
p1 = Person("Mike")
print(p1)                 # >>> Mike(Age: 0)
p2 = Person("Jack", "22") # >>> Age can be integer or float only.
print(p2)                 # >>> Jack(Age: None)

p1.age = 23
print(p1.age)             # >>> 23
p1.age = -1               # >>> Age cannot less than zero.

del(p1.age)
print(p1)                 # >>> Mike(Age: None)

Mike(Age: 0)
Age can be integer or float only.
Jack(Age: None)
23
Age cannot less than zero.
Mike(Age: None)


### Implementation 4:

> Lastly, we will do the exact same thing as above in the most pythonic way, which is using the built-in decorator `@property`

In [4]:
class Person:
    def __init__(self, name, age=0):
        # Initialise Person.name
        if type(name) == str:
            self.name = name
        else:
            print("Person.name can be String only.")
        
        # Initialise Person._age
        self.age = age
        
    def __str__(self):
        return f"{self.name}(Age: {self._age})"
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if type(value) not in (int, float):
            print("Age can be integer or float only.")
            self._age = None
        elif value < 0:
            print("Age cannot less than zero.")
            self._age = None
        else:
            self._age = value
    
    @age.deleter
    def age(self):
        # we don't actually want to delete the age attribute from the memory
        # because in this case, it makes no sense if a person don't have an age attribute
        # so we assume when we use del(person.age)
        # we actually meant to remove the age info from this person
        self._age = None
    
p1 = Person("Mike")
print(p1)                  # >>> Mike(Age: 0)
p2 = Person("Jack", "22")  # >>> Age can be integer or float only.
print(p2)                  # >>> Jack(Age: None)

p1.age = 23
print(p1.age)              # >>> 23
p1.age = -1                # >>> Age cannot less than zero.

del(p1.age)
print(p1)                  # >>> Mike(Age: None)

Mike(Age: 0)
Age can be integer or float only.
Jack(Age: None)
23
Age cannot less than zero.
Mike(Age: None)


### A more complete example

In [5]:
class Person:
    # static variable
    currentYear = 2020

    def __init__(self, name, birthYear=None):
        self.name = name
        self.birthYear = birthYear

    @property
    def name(self):
        print("name getter ran")
        return self._name

    @name.setter
    def name(self, value):
        print("name setter ran")
        if type(value) is str:
            self._name = value
        else:
            raise TypeError("name attribute accepts string only")

    @property
    def birthYear(self):
        print("birthYear getter ran")
        return self._birthYear

    @birthYear.setter
    def birthYear(self, value):
        print("birthYear setter ran")
        if value == None:
            self._birthYear = None
        elif type(value) not in (int, None):
            raise TypeError("birthYear attribute accepts integer only")
        elif value < 1900:
            raise ValueError("birthYear attribute value less than 1900 not supported")
        elif value > Person.currentYear:
            raise ValueError("birthYear cannot be larger than current year")
        else:
            self._birthYear = value

    @birthYear.deleter
    def birthYear(self):
        self._birthYear = None

    @property
    def age(self):
        print("age getter ran")
        if self._birthYear == None:
            return None
        else:
            return Person.currentYear - self._birthYear

    def __str__(self):
        return f"{self.name}(Age: {self.age})"

print("---------------------------------------------")

p1 = Person("Mark", 1997)
print(f"p1: {p1}")

print("---------------------------------------------")

p1.birthYear = 1999
print(f"p1: {p1}")

print("---------------------------------------------")

p2 = Person("Meridith")
print(f"p2: {p2}")

print("---------------------------------------------")

p2.birthYear = 1989
print(f"p2: {p2}")

print("---------------------------------------------")

try:
    p2.age = 30
except AttributeError:
    print("Attribute Error catched as expected.")
else:
    print("!!! something is wrong")

print("---------------------------------------------")

try:
    p = Person(12, 1997)
except TypeError:
    print("Type Error catched as expected.")
else:
    print("!!! something is wrong")

print("---------------------------------------------")

try:
    p = Person("Jackson", "1999")
except TypeError:
    print("Type Error catched as expected.")
else:
    print("!!! something is wrong")

print("---------------------------------------------")

try:
    p = Person("Jackson", 100)
except ValueError:
    print("Value Error catched as expected.")
else:
    print("!!! something is wrong")

print("---------------------------------------------")

try:
    p = Person("Jackson", 2070)
except ValueError:
    print("Value Error catched as expected.")
else:
    print("!!! something is wrong")

---------------------------------------------
name setter ran
birthYear setter ran
name getter ran
age getter ran
p1: Mark(Age: 23)
---------------------------------------------
birthYear setter ran
name getter ran
age getter ran
p1: Mark(Age: 21)
---------------------------------------------
name setter ran
birthYear setter ran
name getter ran
age getter ran
p2: Meridith(Age: None)
---------------------------------------------
birthYear setter ran
name getter ran
age getter ran
p2: Meridith(Age: 31)
---------------------------------------------
Attribute Error catched as expected.
---------------------------------------------
name setter ran
Type Error catched as expected.
---------------------------------------------
name setter ran
birthYear setter ran
Type Error catched as expected.
---------------------------------------------
name setter ran
birthYear setter ran
Value Error catched as expected.
---------------------------------------------
name setter ran
birthYear setter ran
Val

### Understand the Implementation of `property`

[To see how property() is implemented in terms of the descriptor protocol, here is a pure Python equivalent](https://docs.python.org/3/howto/descriptor.html)

```
class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

```